# Closures in Julia

Many languages support closures. Today I needed to run part of my program for up to an hour.
Instead of putting the timer in the same block of code, I wanted to separate the responsibility. Closures seemed like a good idea.

## Dummy closure


In [8]:
function withDummy(func::Function)
    println("[DEBUG] Before")
    func()
    println("[DEBUG] After")
end

withDummy (generic function with 1 method)

In [9]:
withDummy() do
    println("Hi")
end

[DEBUG] Before
Hi
[DEBUG] After


## Conclusion:

Great. This works like I wanted. Now I'm going to work on the concurrent timer thing